In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# getting data

In [0]:
import bs4 as bs
import datetime as dt
import os
import pickle
import requests

import pandas as pd
import pandas_datareader.data as pdreader
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

In [0]:
def get_1stock_data_from_yahoo(ticker):
  #tickers = save_sp500_tickers()
  start = dt.datetime(2010, 1, 1)
  end = dt.datetime.now()

  main_df = pd.DataFrame()

  df = pdreader.DataReader(ticker, 'yahoo', start, end)
      #df.reset_index(inplace = True)
  #df.rename(columns={'Adj Close': ticker}, inplace=True)
  #df.drop(['Open', 'High', 'Low', 'Close', 'Volume','Adj Close'], 1, inplace=True)


  print(df.head())
  print(df.tail())
  #main_df.to_csv('sp500_joined_closes.csv')
  
  return df
  
    

In [0]:
target_stock = ['AAPL', 'MSFT','GOOGL','AMZN','IBM']

In [23]:
AAPL_df = get_1stock_data_from_yahoo('AAPL')

                 High        Low       Open      Close       Volume  Adj Close
Date                                                                          
2010-01-04  30.642857  30.340000  30.490000  30.572857  123432400.0  26.782711
2010-01-05  30.798571  30.464285  30.657143  30.625713  150476200.0  26.829010
2010-01-06  30.747143  30.107143  30.625713  30.138571  138040000.0  26.402260
2010-01-07  30.285715  29.864286  30.250000  30.082857  119282800.0  26.353460
2010-01-08  30.285715  29.865715  30.042856  30.282858  111902700.0  26.528664
                  High         Low  ...      Volume   Adj Close
Date                                ...                        
2019-07-22  207.229996  203.610001  ...  22277900.0  207.220001
2019-07-23  208.910004  207.289993  ...  18355200.0  208.839996
2019-07-24  209.149994  207.169998  ...  14991600.0  208.669998
2019-07-25  209.240005  206.729996  ...  13909600.0  207.020004
2019-07-26  209.729996  207.139999  ...  17601200.0  207.740005

In [0]:
datetime.datetime(2018, 8, 5)

In [28]:
AAPL_learndf = AAPL_df.ix[(AAPL_df.index >= '2013-01-01') & (AAPL_df.index <= '2018-01-01')]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [0]:
AAPL_df.to_csv('data_aapl')


# Trading

In [1]:
pip install backtrader

In [3]:
pip install backtrader[plotting]

     |████████████████████████████████| 419kB 2.8MB/s 


In [0]:
import backtrader as bt
class MyStrategy(bt.Strategy):

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=15)

    def next(self):
        if self.sma > self.data.close:
            # Do something
            pass

        elif self.sma < self.data.close:
            # Do something else
            pass

In [0]:
cerebro.broker.setcommission(commission=0.001) 

In [37]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import os.path
import sys
import backtrader as bt

class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=15)
        self.rsi = bt.indicators.RelativeStrengthIndex()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        print('rsi:', self.rsi[0])
        if self.order:
            return

        if not self.position:
            if (self.rsi[0] < 30):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy(size=500)

        else:
            if (self.rsi[0] > 70):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell(size=500)


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    cerebro.broker.setcommission(commission=0.001)
#/content/data_aapl
    datapath = 'data_aapl'

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        fromdate=datetime.datetime(2013, 1, 1),
        todate=datetime.datetime(2018, 8, 5),
        reverse=True)

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot()

Starting Portfolio Value: 100000.00


IndexError: ignored